In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import logging
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import random
from aiohttp import ClientSession
import asyncio

In [ ]:

async def handle_response_error(response):
    if response.status == 429:
        logging.error("Yêu cầu thất bại, mã lỗi 429 (Quá nhiều yêu cầu), đang thử lại sau 10 giây...")
        await asyncio.sleep(random.uniform(5, 7))
    elif response.status == 403:
        logging.error("Yêu cầu thất bại, mã lỗi 403 (Quá nhiều yêu cầu), đang thử lại sau 10 giây...")
        await asyncio.sleep(random.uniform(5, 7))
    else:
        print(f"Yêu cầu thất bại, mã lỗi: {response.status}")

async def fetch_all_players_data(session, player_links):
    semaphore = asyncio.Semaphore(6)  # Số lượng yêu cầu đồng thời
    tasks = []
    async def fetch_with_semaphore(url):
        async with semaphore:
            try:
                return await crawl_player_details(session, url)
            except ClientError as e:
                print(f"Error for URL {url}: {e}")
                return None
    
    for player_link in player_links:
        url = f"https://sofifa.com{player_link['href']}"
        if "/player/" in url and "/random" not in url:
            task = fetch_with_semaphore(url)
            tasks.append(task)
            
    return await asyncio.gather(*tasks)

async def crawl_all_players_data(year, file_path='D:/Jupyter/DATA/test4.csv'):
    base_url = f"https://sofifa.com/players?r={year}&set=true"
    #scraper = cloudscraper.create_scraper()
    data_frame = pd.DataFrame()
    page = 1
    has_players = True
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=1000)
    async with ClientSession() as session:
        #player_data_list = []
        while has_players:
            page_url = f"{base_url}&offset={((page - 1) * 61)}"
            headers = {'User-Agent': user_agent_rotator.get_random_user_agent()}
            response = await session.get(page_url, headers=headers)
            #time.sleep(3)
            if response.status == 200:
                soup = BeautifulSoup(await response.text(), 'html.parser')
                player_links = soup.find_all('a', href=lambda href: href and "/player/" in href)

                if not player_links:
                    has_players = False
                    break
                else:
                    data_frames = await fetch_all_players_data(session, player_links)
                    
                    for player_data in data_frames:
                        if player_data is not None:
                            df = pd.DataFrame([player_data])
                            file_exists = os.path.isfile(file_path)
                            df.to_csv(file_path, mode='a', header=not file_exists, index=False)
                    page += 1
                    #await asyncio.sleep(random.uniform(1, 3))  # Thêm thời gian chờ giữa các trang
            else:
                await handle_response_error(response)
    
    return data_frame

In [ ]:
async def crawl_player_details(session, player_url):
    scraper = cloudscraper.create_scraper(
        browser={
        'browser': 'chrome',
        'platform': 'windows',
        'desktop': True,
        'mobile': False,
        'custom':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 Edg/118.0.2088.46"
    }, delay = 5)
    response = await session.get(player_url)
    await asyncio.sleep(random.uniform(1, 3))
    if response.status == 200:
        soup = BeautifulSoup(await response.text(), 'html.parser')
        short_name = soup.find('h1', class_='ellipsis').text.strip()
        player_info = soup.find('div', class_='info')
        player_name = player_info.find('h1').text.strip()
        nationality = player_info.find('a', title=True)['title'] or "Không xác định"
        #Lấy thông tin chuỗi thông tin cơ bản
        meta_info = player_info.find('div', class_='meta ellipsis').text.strip()
        # Sử dụng biểu thức chính quy để trích xuất thông tin
        match = re.match(r'(.+?) (\d+)y\.o\. \((\w+ \d+, \d+)\) (\d+)cm / (\S+) (\d+)kg', meta_info)
        position = match.group(1)
        age = match.group(2)
        birthday = match.group(3)
        height = match.group(4)
        weight_unit = match.group(5)
        weight = match.group(6)
        player_info2 = soup.find('section', class_='card spacing')
        overall_rating = player_info2.find('div', string='Overall rating').find_previous('span').text.strip()
        potential = player_info2.find('div', string='Potential').find_previous('span').text.strip()
        value = player_info2.find('div', string='Value').previous_sibling.strip()
        wage = player_info2.find('div', string='Wage').previous_sibling.strip()
        #list chứa thông tin cầu thủ
        player_data = [
            ('Player URL', player_url),
            ('Short Name', short_name),
            ('Player Name', player_name),
            ('Nationality', nationality),
            ('Position', position),
            ('Age', age),
            ('Birthday', birthday),
            ('Height', height),
            ('Weight', weight),
            ('Overall Rating', overall_rating),
            ('Potential', potential),
            ('Value', value),
            ('Wage', wage),
            
        ]
        profile_divs = soup.find_all('div', class_='card')
        # Duyệt qua mỗi phần tử div 'card' và lấy dữ liệu
        for profile_div in profile_divs:
            
            ul_element_profile = profile_div.find('ul', class_='pl')
            
            # Kiểm tra xem thẻ <ul class="pl"> có tồn tại không
            if ul_element_profile:
                # Tìm tất cả các phần tử <li> trong thẻ <ul class="pl">
                li_elements_profile = ul_element_profile.find_all('li')
                
                for li in li_elements_profile:
                    # Lấy các thẻ con bên trong <li>
                    label_tag = li.find('label')
                    span_tag_profile = li.find('span')
                    has_svg = bool(li.find('svg'))
                    if label_tag:
                        attribute_name_profile = label_tag.text.strip()
                        
                        if span_tag_profile:
                            # Trường hợp có <span>, lấy giá trị từ <span>
                            attribute_value_profile = span_tag_profile.text.strip()
                        elif not has_svg:
                            # Trường hợp không có <span> và không có <svg>, lấy giá trị từ toàn bộ <li>
                            attribute_value_profile = li.text.strip().replace(attribute_name_profile, '')
                        else:
                            # Trường hợp có <svg>
                            attribute_value_profile = li.text.strip().replace(attribute_name_profile, '')
                            # Sử dụng biểu thức chính quy để lấy con số trước <svg>
                            match1 = re.search(r'\d+', attribute_value_profile)
                            attribute_value_profile = match1.group() if match1 else ""
                        
                        #print(f"{attribute_name_profile} {attribute_value_profile}")
                        player_data.append((attribute_name_profile, attribute_value_profile))
            h5_element = profile_div.find('h5')
            if h5_element:
                a_element = h5_element.find('a')
                if a_element:
                    club_link = a_element['href']
                    match = re.search(r'/team/\d+/(.*)/', club_link)
                    if match:
                        club_name = match.group(1)
                        #print(f"Club Name: {club_name}")
                        player_data.append(club_name)
            else:
                # Nếu không tìm thấy thẻ <ul class='pl'> trong thẻ div với class='card', bỏ qua và không in gì cả
                pass   
        
        card_divs = soup.find_all('div', class_='card')
        for card_div in card_divs:
            # Tìm thẻ <ul class="pl">
            ul_element = card_div.find('ul', class_='pl')
            # Kiểm tra xem thẻ <ul class="pl"> có tồn tại không
            if ul_element:
                # Tìm tất cả các phần tử <li> trong thẻ <ul class="pl">
                li_elements = ul_element.find_all('li')
                
                # Trích xuất dữ liệu và in ra màn hình
                for li in li_elements:
                    span_tag = li.find('span', class_='bp3-tag')
                    attribute_name_span = li.find('span', role='tooltip')

                    # Kiểm tra xem cả hai thẻ có tồn tại không trước khi truy xuất text
                    if span_tag and attribute_name_span:
                        attribute_value = span_tag.text.strip()
                        attribute_name = attribute_name_span.text.strip() 
                        #print(f"{attribute_name} {attribute_value}")
                        player_data.append((attribute_name, attribute_value))
                    else:
                        # Nếu một số phần tử bị thiếu, bỏ qua và không in gì cả
                        pass      
            else:
                # Nếu không tìm thấy thẻ <ul class='pl'> trong thẻ div với class='card', bỏ qua và không in gì cả
                pass
        
        print(player_data)
        return player_data
    else:
        await handle_response_error(response)
        return None  # Trả về None nếu có lỗi

In [ ]:
# final_df = pd.DataFrame()
# final_df = pd.concat([final_df, crawl_all_players_data(220069, num_threads=8)], ignore_index=True)
#print(data_frame)
await crawl_all_players_data(220069)

In [2]:
import pandas as pd
data_frame = pd.read_csv('D:/Jupyter/DATA/Final_Crawl.csv',header=None)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18736\2492001212.py:2: DtypeWarning: Columns (28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frame = pd.read_csv('D:/Jupyter/DATA/Final_Crawl.csv',header=None)


In [3]:
pd.set_option('display.max_columns', 100)
#pd.set_option('display.max_rows', 100)
data_frame.head(10)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
0,PLAYER_URL,Short Name,Player Name,Nationality,Player_Position,Age,Birthday,Height(cm),Weight(kg),Overall Rating,Potential,Value,Wage,Preferred foot,Skill moves,Weak foot,International reputation,Work rate,Body type,Real face,Release clause,ID,Club_Position,Club_Kit number,Joined,Loaned,Contract valid until,Club_name,National_Position,National_Kit number,National_name,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
1,"('Player URL', 'https://sofifa.com/player/2326...","('Short Name', 'S. Singh')","('Player Name', 'Sarpreet Singh')","('Nationality', 'New Zealand')","('Position', 'CAM RM')","('Age', '22')","('Birthday', 'Feb 20, 1999')","('Height', '177')","('Weight', '70')","('Overall Rating', '69')","('Potential', '78')","('Value', '€3.1M')","('Wage', '€5K')","('Preferred foot', 'Left')","('Skill moves', '3')","('Weak foot', '3')","('International reputation', '1')","('Work rate', 'High/ Medium')","('Body type', 'Lean (170-185)')","('Real face', 'No')","('Release clause', '€5.4M')","('ID', '232669')","('Position', 'LAM')","('Kit number', '15')","('Joined', 'Jul 1, 2019')",NaN,"('Contract valid until', '2024')",jahn-regensburg,"('Position', 'CAM')","('Kit number', '10')",new-zealand,"('Crossing', '67')","('Finishing', '64')","('Heading accuracy', '44')","('Short passing', '70')","('Volleys', '60')","('Dribbling', '71')","('Curve', '73')","('FK Accuracy', '64')","('Long passing', '69')","('Ball control', '70')","('Acceleration', '80')","('Sprint speed', '70')","('Agility', '78')","('Reactions', '66')","('Balance', '81')","('Shot power', '68')","('Jumping', '53')","('Stamina', '66')","('Strength', '51')","('Long shots', '67')","('Aggression', '63')","('Interceptions', '51')","('Positioning', '60')","('Vision', '72')","('Penalties', '64')","('Composure', '64')","('Defensive awareness', '51')","('Standing tackle', '47')","('Sliding tackle', '41')","('GK Diving', '14')","('GK Handling', '6')","('GK Kicking', '12')","('GK Positioning', '8')","('GK Reflexes', '8')"
2,"('Player URL', 'https://sofifa.com/player/2382...","('Short Name', 'D. Gazdag')","('Player Name', 'Dániel Gazdag')","('Nationality', 'Hungary')","('Position', 'CAM CM')","('Age', '25')","('Birthday', 'Mar 2, 1996')","('Height', '178')","('Weight', '76')","('Overall Rating', '69')","('Potential', '72')","('Value', '€1.8M')","('Wage', '€4K')","('Preferred foot', 'Right')","('Skill moves', '3')","('Weak foot', '3')","('International reputation', '1')","('Work rate', 'High/ Medium')","('Body type', 'Normal (170-185)')","('Real face', 'No')","('Release clause', '€2.8M')","('ID', '238277')","('Position', 'SUB')","('Kit number', '15')","('Joined', 'May 11, 2021')",NaN,"('Contract valid until', '2022')",philadelphia-union,"('Position', 'CAM')","('Kit number', '6')",hungary,"('Crossing', '51')","('Finishing', '66')","('Heading accuracy', '47')","('Short passing', '70')","('Volleys', '59')","('Dribbling', '72')","('Curve', '61')","('FK Accuracy', '60')","('Long passing', '68')","('Ball control', '71')","('Acceleration', '73')","('Sprint speed', '78')","('Agility', '71')","('Reactions', '68')","('Balance', '67')","('Shot power', '64')","('Jumping', '59')","('Stamina', '65')","('Strength', '59')

In [4]:

import ast

# Đọc DataFrame từ file CSV

# Hàm xử lý để loại bỏ key và giữ lại giá trị
def process_data(x):
    try:
        return ast.literal_eval(x)[1]
    except:
        return x

# Áp dụng hàm xử lý cho toàn bộ DataFrame
df = data_frame.applymap(process_data)

# Xuất ra CSV hoặc hiển thị DataFrame
df.to_csv('output1.csv', index=False, encoding='utf-8-sig')  # Xuất DataFrame ra file CSV
print(df)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18736\853689266.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = data_frame.applymap(process_data)


                                                      0               1   \
0                                             PLAYER_URL      Short Name   
1      https://sofifa.com/player/232669/sarpreet-sing...        S. Singh   
2      https://sofifa.com/player/238277/daniel-gazdag...       D. Gazdag   
3      https://sofifa.com/player/186561/aaron-ramsey/...       A. Ramsey   
4      https://sofifa.com/player/236547/jan-sykora/22...       J. Sýkora   
...                                                  ...             ...   
18838  https://sofifa.com/player/263248/ioannis-tsout...   I. Tsoutsouki   
18839  https://sofifa.com/player/140814/stavros-georg...     S. Georgiou   
18840  https://sofifa.com/player/263249/stylianos-vro...      S. Vrontis   
18841  https://sofifa.com/player/262563/marios-kokkin...  M. Kokkinoftas   
18842  https://sofifa.com/player/267767/stefanos-kitt...       S. Kittos   

                       2               3                4    5             6   \
0     

In [8]:
df.to_csv('test.csv')